In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv('bank_data.csv')

In [3]:
df.head()

,year,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A56,A57,A58,A59,A60,A61,A62,A63,A64,class
0,3.0,0.174190,0.41299,0.14371,1.3480,-28.9820,0.60383,0.219460,1.1225,1.1961,...,0.163960,0.375740,0.83604,0.000007,9.7145,6.2813,84.291,4.3303,4.0341,0.0
1,3.0,0.146240,0.46038,0.28230,1.6294,2.5952,0.00000,0.171850,1.1721,1.6018,...,0.027516,0.271000,0.90108,0.000000,5.9882,4.1103,102.190,3.5716,5.9500,0.0
2,3.0,0.000595,0.22612,0.48839,3.1599,84.8740,0.19114,0.004572,2.9881,1.0077,...,0.007639,0.000881,0.99236,0.000000,6.7742,3.7922,64.846,5.6287,4.4581,0.0
3,3.0,0.024526,0.43236,0.27546,1.7833,-10.1050,0.56944,0.024526,1.3057,1.0509,...,0.048398,0.043445,0.95160,0.142980,4.2286,5.0528,98.783,3.6950,3.4844,0.0
4,3.0,0.188290,0.41504,0.34231,1.9279,-58.2740,0.00000,0.233580,1.4094,1.3393,...,0.176480,0.321880,0.82635,0.073039,2.5912,7.0756,100.540,3.6303,4.6375,0.0


In [4]:
X = df.drop(['class','year'], axis=1)
y = df['class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the XGBClassifier
xgb = XGBClassifier(eval_metric='logloss')

# Setup the Sequential Feature Selector
sfs = SequentialFeatureSelector(xgb, n_features_to_select=10, direction='backward', scoring='accuracy', cv=5)

# Create a pipeline that first selects features and then fits the classifier
pipeline = Pipeline([
    ('feature_selection', sfs),
    ('classification', xgb)
])

# Evaluate the pipeline using cross-validation
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

print("CV Scores:", scores)
print("Average CV Score:", np.mean(scores))

# Fit the pipeline on the entire training set
pipeline.fit(X_train, y_train)

# Selected features based on the entire training set
selected_features = X_train.columns[pipeline.named_steps['feature_selection'].get_support()]

print("Selected features:", selected_features)


CV Scores: [0.95716835 0.96192742 0.95892857 0.96130952 0.96488095]
Average CV Score: 0.9608429647885328
Selected features: Index(['A5', 'A17', 'A26', 'A27', 'A32', 'A33', 'A34', 'A39', 'A46', 'A58'], dtype='object')
